In [77]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from lxml import etree

def find_upmost_parent(element, parent_list):
    if element.tag in parent_list:
        res = element.tag
    elif element.getparent().tag in parent_list:
        res = element.getparent().tag
    else:
        res = find_upmost_parent(element.getparent(), parent_list)
    return res

def isrepresented_by_children(disease, disease_group_tree, disease_res_df, column_name='disease'):
    children_diseases = set([item.tag for item in disease_group_tree.getchildren()])
    affected_individual_num = disease_res_df['Disease_number_within_Drug_Use'][disease_res_df[column_name]==disease].values[0]
    children_affected_individual_num = 0
    for child_disease in children_diseases:
        if child_disease in disease_res_df[column_name].unique():
            children_affected_individual_num += disease_res_df['Disease_number_within_Drug_Use'][disease_res_df[column_name]==child_disease].values[0]
    if children_affected_individual_num >= affected_individual_num*0.8:
        print(f"{disease} is fully represnted by its children.")
        return True
    else:
        return False

In [81]:
# read from the xml file
disease_et = etree.parse("../data/anomalies_ontology.xml")
drug_et = etree.parse("../data/medication_atccodes_ontology.xml")
res_df = pd.read_csv('../results/allcombinations_atccodes_results.tsv', sep='\t')
res_othermedication = pd.read_csv('../results/allcombinations_othermedications_results.tsv', sep='\t')
print(res_df.shape, res_othermedication.shape)

(5939, 10) (1140, 10)


Filtering criteria:
1. More than 10 individuals' mother take this medication and have this anomaly
2. p-value (after FDR-BH correction) below 0.05

In [82]:
res_df = res_df[res_df['Disease_number_within_Drug_Use']>5]
res_df['corrected_p'] = multipletests(res_df['p'], method='fdr_bh')[1]
sig_res_df = res_df[res_df['corrected_p']<0.05]
print(sig_res_df.shape)

res_othermedication = res_othermedication[res_othermedication['Disease_number_within_Drug_Use']>10]
res_othermedication['corrected_p'] = multipletests(res_othermedication['p'], method='fdr_bh')[1]
sig_res_othermedication = res_othermedication[res_othermedication['corrected_p']<0.05]
print(sig_res_othermedication.shape)

sig_res_df.sort_values(by=['corrected_p']).to_csv('../results/significant_atccodes_results.tsv',
                                                                 sep='\t')
sig_res_othermedication.sort_values(by=['corrected_p']).to_csv('../results/significant_othermedications_results.tsv',
                                                                 sep='\t')

(3, 11)
(3, 11)


Pruning criteria:
1.  If the children objects have more than 80% individuals, then its parent is discarded

In [69]:
# add the most upper parent to each medication & anomaly
disease_root = disease_et.getroot()
parent_list = [item.tag for item in disease_root.getchildren()]
sig_res_df['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in sig_res_df['disease']]
sig_res_othermedication['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in sig_res_othermedication['disease']]

drug_root = drug_et.getroot()
drug_parent_list = [item.tag for item in drug_root.getchildren()]
sig_res_df['upmost_drug_parent'] = [find_upmost_parent(drug_et.xpath(f'//{drug}')[0], drug_parent_list) 
                                   for drug in sig_res_df['drug']]

/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [72]:
res_df_noduplicated_diseases

,Drug_use_number,Disease_number_within_Drug_Use,Disease_ratio_within_Drug_Use,p,expected_disease_ratio,odds_ratio,odds_ratio_ci_lower,odds_ratio_ci_higher,disease,drug,corrected_p,upmost_disease_parent,upmost_drug_parent,is_represented_by_children_diseases
184,54,7,0.129630,2.038212e-07,1.107692,7.406750,3.292029,16.664475,spinabif,N03T1,1.361525e-04,nervsyst,N03T1,False
5771,54,8,0.148148,7.556177e-187,0.065587,1279.130435,156.799225,10434.839016,terasyn,N03T1,7.571289e-184,terasyn,N03T1,False
5790,54,9,0.166667,6.833670e-240,0.065587,2942.400000,168.036130,51522.953856,valproate,N03T1,1.369467e-236,valproate,N03T1,False


In [73]:
res_df_noduplicated_diseases = pd.DataFrame()
# disease_group = 'nervsyst'
for disease_group in sig_res_df['upmost_disease_parent'].unique():
    drug_list = sig_res_df['drug'][(sig_res_df['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = sig_res_df[(sig_res_df['upmost_disease_parent']==disease_group) & 
                                   (sig_res_df['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        res_df_noduplicated_diseases = pd.concat([res_df_noduplicated_diseases, subset_res_df], axis=0)

sig_res_othermedication_noduplicated_diseases = pd.DataFrame()
# disease_group = 'nervsyst'
for disease_group in sig_res_othermedication['upmost_disease_parent'].unique():
    drug_list = sig_res_othermedication['drug'][(sig_res_othermedication['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = sig_res_othermedication[(sig_res_othermedication['upmost_disease_parent']==disease_group) & 
                                                (sig_res_othermedication['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        sig_res_othermedication_noduplicated_diseases = pd.concat([sig_res_othermedication_noduplicated_diseases, 
                                                                   subset_res_df], axis=0)
    

res_df_uniquecombi = pd.DataFrame()
for drug_group in res_df_noduplicated_diseases['upmost_drug_parent'].unique():
    for disease in res_df_noduplicated_diseases['disease'][res_df_noduplicated_diseases['upmost_drug_parent']==drug_group].unique():
        subset_res_df = res_df_noduplicated_diseases[(res_df_noduplicated_diseases['upmost_drug_parent']==drug_group) & 
                                                     (res_df_noduplicated_diseases['disease']==disease)]
        subset_res_df['is_represented_by_children_drugs'] = [isrepresented_by_children(drug, 
                                                                                       drug_et.xpath(f'//{drug}')[0], 
                                                                                       subset_res_df,
                                                                                       'drug')
                                                             for drug in subset_res_df['drug']]
        num = subset_res_df[subset_res_df['is_represented_by_children_drugs']].shape[0]
        if num > 0:
            print(f"Deleting {num} drug that are fully represented in its children.")
        res_df_uniquecombi = pd.concat([res_df_uniquecombi,
                                       subset_res_df[subset_res_df['is_represented_by_children_drugs']==False]],
                                       axis=0)

/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [74]:
# save pruned results
import os
if not os.path.isdir('../results'):
    os.mkdir('../results')
res_df_uniquecombi.sort_values(by=['upmost_disease_parent', 
                                   'upmost_drug_parent']).to_csv('../results/pruned_atccodes_results.tsv',
                                                                 sep='\t')

sig_res_othermedication_noduplicated_diseases.sort_values(by=['upmost_disease_parent']).to_csv('../results/pruned_othermedications_results.tsv',
                                                                 sep='\t')